### Построение и программная реализация алгоритма сплайн-интерполяции табличных функций

In [1]:
class Point:
    def __init__(self, x: float = 0, y: float = 0) -> None:
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"({self.x:.3}, {self.y:.3})"


class Points:
    def __init__(self, init_data: list[Point] = []) -> None:
        self.data: list[Point] = init_data

    def add_point(self, p: Point):
        self.data.append(p)

    def parse_table(self, t: list[list[float]]):
        for i in range(len(t[0])):
            self.data.append(Point(t[0][i], t[1][i]))

    def find_interval(self, x: float, length: int) -> {int, int}:
        closest_i = 0, 99999999999999
        for i, p in enumerate(self.data):
            if abs(p.x - x) < closest_i[1]:
                closest_i = i, abs(p.x - x)

        i1: int
        i2: int

        if closest_i[0] >= length // 2:
            i1 = closest_i[0] - length // 2
        else:
            i1 = 0

        if len(self.data) > i1 + length:
            i2 = i1 + length
        else:
            i1 = len(self.data) - length
            i2 = i1 + length

        return i1, i2

    def get_interval_slice(self, x, length) -> list[Point]:
        i1, i2 = self.find_interval(x, length)
        return self.data[i1:i2]

    def get_dx_list(self) -> list[float]:
        res = []
        pprev = False
        for p in self.data:
            if pprev:
                res.append(p.x - pprev.x)
                pprev = p
            else:
                pprev = p

        return res

    def find_closest_point(self, x) -> {int, Point}:
        closest_i = 0, 99999999999999
        for i, p in enumerate(self.data):
            if abs(p.x - x) < closest_i[1]:
                closest_i = i, abs(p.x - x)

        return closest_i[0], self.data[closest_i[0]]

    def __str__(self) -> str:
        res = ""
        res += "x:\t"
        for p in self.data:
            res += f"{p.x:.3f}\t"
        res += "\n"
        res += "y:\t"
        for p in self.data:
            res += f"{p.y:.3f}\t"
        return res

In [2]:
class CubicPolynome:
    def __init__(self, a: float, b: float, c: float, d: float, x0: float) -> None:
        self.a: float = a
        self.b: float = b
        self.c: float = c
        self.d: float = d
        self.x0: float = x0

    def get(self, x) -> float:
        xoff = x - self.x0

        return self.a * (xoff**3) + self.b * (xoff**2) + self.c * xoff + self.d

In [3]:
class CubicSpline:
    def __init__(self, c_0: float = 0, c_n: float = 0) -> None:
        self.c_0 = c_0
        self.c_n = c_n

    def thomas(self, pts: Points, x: float) -> CubicPolynome:
        h = pts.get_dx_list()
        data = pts.data

        if len(data) == 2:
            return CubicPolynome(
                (self.c_n - self.c_0) / h[0] - 1 /
                3 * h[0] * (2 * self.c_0 + self.c_n),
                self.c_n,
                (data[1].y - data[0].y) / (3 * h[0]),
                data[0].y,
                data[0].x,
            )

        ksi = [0]
        eta = [self.c_0 / 2]

        for i in range(1, len(h)):
            a = h[i - 1]
            b = -2 * (h[i - 1] + h[i])
            d = h[i]
            f = -3 * (
                (data[i + 1].y - data[i].y) / h[i]
                - (data[i].y - data[i - 1].y) / h[i - 1]
            )

            ksi.append(d / (b - a * ksi[i - 1]))
            eta.append((a * eta[i - 1] + f) / (b - a * ksi[i - 1]))

        n = len(data)
        c = [0 for _ in range(n)]
        c[0] = self.c_0 / 2
        c[n - 1] = self.c_n / 2

        for i in range(len(ksi) - 1, 0, -1):
            c[i] = ksi[i] * c[i + 1] + eta[i]

        i, pt = pts.find_closest_point(x)
        return CubicPolynome(
            (c[i + 1] - c[i]) / (3.0 * h[i]),
            c[i + 1],
            (data[i + 1].y - data[i].y) / h[i]
            - 1.0 / 3.0 * h[i] * (c[i + 1] + 2.0 * c[i]),
            data[i].y,
            pt.x,
        )

In [4]:
def newton_2nd_deg(pts: Points, x: float) -> float:
    data = pts.get_interval_slice(x, 3)
    dds = []
    for pt in data:
        dds.append(pt.y)

    for i in [1, 2]:
        for j in range(2, i - 1, -1):
            dds[j] = (dds[j] - dds[j - 1]) / (data[j].x - data[j - i].x)

    return 2 * dds[2]


def newton_interpolate(x: float, deg: int, pts: Points):
    data = pts.get_interval_slice(x, deg + 1)

    coeff_table: list[list[float]] = [
        [s.x for s in data],
        [s.y for s in data],
    ]

    for _ in range(deg):
        coeff_table.append([])

    for i in range(deg):
        for j in range(deg - i):
            coeff_table[i + 2].append(
                (coeff_table[i + 1][j] - coeff_table[i + 1][j + 1])
                / (coeff_table[0][j] - coeff_table[0][j + i + 1])
            )

    res = coeff_table[1][0]
    xcoef: float = 1.0

    for i in range(deg):
        xcoef *= x - coeff_table[0][i]
        res += coeff_table[i + 2][0] * xcoef

    return res


def newton_interpolate_2(x: float, deg: int, pts: Points):
    data = pts.get_interval_slice(x, deg + 1)
    dds = []
    for pt in data:
        dds.append(pt.y)

    res = dds[0]
    term = 1.0

    for i in range(1, deg + 1):
        for j in range(deg, i - 1, -1):
            dds[j] = (dds[j] - dds[j - 1]) / (data[j].x - data[j - i].x)

        term *= x - data[i - 1].x
        res += term * dds[i]

    return res

In [5]:
table1: list[list[float]] = [
    [0, 0.5, 0.75, 1.25, 1.5, 2, 3, 4, 5, 6, 7],
    [0.250, 0.571, 0.842, 0.842, 0.571, 0.250, 0.077, 0.036, 0.020, 0.013, 0.009],
]

table2: list[list[float]] = [
    [-0.5, 0, 2, 3, 4],
    [-0.75, -1, 3, 8, 15],
    [-1, 0, 4, 6, 8],
]

table3: list[list[float]] = [
    [0, 0.25, 0.5, 0.75, 1],
    [1, 0.924, 0.707, 0.383, 0],
]

table = table1


pts = Points()
pts.parse_table(table)
x: float = 3.33

left = newton_2nd_deg(pts, pts.data[0].x)
right = newton_2nd_deg(pts, pts.data[-1].x)

print("points:")
print(pts)
print()

print(f"target x: {x}")
print()

print("spline none:")
print(CubicSpline().thomas(pts, x).get(x))
print()

print("spline left:")
print(CubicSpline(c_0=left).thomas(pts, x).get(x))
print()

print("spline both:")
print(CubicSpline(c_0=left, c_n=right).thomas(pts, x).get(x))
print()

print("newton:")
print(newton_interpolate(x, min(4, len(pts.data)), pts))

points:
x:	0.000	0.500	0.750	1.250	1.500	2.000	3.000	4.000	5.000	6.000	7.000	
y:	0.250	0.571	0.842	0.842	0.571	0.250	0.077	0.036	0.020	0.013	0.009	

target x: 3.33

spline none:
0.05750241793835335

spline left:
0.0574756115612994

spline both:
0.05747345787603183

newton:
0.06546325335000014
